# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning

# Feature Engineering

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

En este notebook se buscan atributos nuevos para concatenar al set de datos original, así pudiendo armar un modelo predictivo más robusto y eficiente.

In [ ]:
import pandas as pd
import numpy as np
import calendar

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Setting random seed.
seed = 42

In [ ]:
df_events = pd.read_csv('./data/events_up_to_01062018.csv', low_memory=False)
df_sessions = pd.read_csv('./data/sessions.csv', low_memory=False)
df_brands = pd.read_csv('data/brands.csv')
df_os = pd.read_csv('data/os.csv')
df_browsers = pd.read_csv('data/browsers.csv')

df = df_events.merge(df_sessions, how='left', left_index=True, right_index=True)
df = df.merge(df_browsers, how='left', on='browser_version')
df = df.merge(df_os, how='left', on='operating_system_version')
df = df.merge(df_brands, how='left', on='model')

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['month_number'] = df['timestamp'].dt.month

df['is_conversion'] = df['event'] == 'conversion'
df['is_checkout'] = df['event'] == 'checkout'

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.head()

### 1 - user-features

Checkouts y conversiones por usuario

In [ ]:
# Total checkouts and conversions
udf_tmp1 = df.groupby('person').agg({'is_checkout':'sum', 
                                     'is_conversion':'sum', 
                                     'session_total_events':'sum',
                                     'session_first':'sum',                                                                         'session_total_conversions':'sum',
                                     'session_total_checkouts':'sum',
                                     'session_total_conversions':'sum'})

udf_tmp1.columns = ['total_checkouts',
                    'total_conversions',
                    'total_events',
                    'total_sessions',
                    'total_session_checkouts',
                    'total_session_conversions']
udf_tmp1['total_checkouts'] = udf_tmp1['total_checkouts'].astype('int')
udf_tmp1['total_conversions'] = udf_tmp1['total_conversions'].astype('int')
udf_tmp1['total_events'] = udf_tmp1['total_events'].astype('int')
udf_tmp1['total_sessions'] = udf_tmp1['total_sessions'].astype('int')
udf_tmp1['total_session_checkouts'] = udf_tmp1['total_session_checkouts'].astype('int')
udf_tmp1['total_session_conversions'] = udf_tmp1['total_session_conversions'].astype('int')

udf_tmp1['avg_events_per_session'] = udf_tmp1['total_events'] / udf_tmp1['total_sessions']

udf_tmp1['has_checkout'] = udf_tmp1['total_checkouts'] > 0
udf_tmp1['has_conversion'] = udf_tmp1['total_conversions'] > 0

udf_tmp1 = udf_tmp1.astype('int') 

In [ ]:
display(len(udf_tmp1))
udf_tmp1[udf_tmp1['total_conversions'] > 10].sort_values('total_conversions', ascending=False)

### 2 - user-monthly-features

Checkouts y conversiones por usuario por mes

In [ ]:
udf_tmp2 = df['person'].drop_duplicates().to_frame().set_index('person')
display(udf_tmp2.head())

for i in range(1,6):
    gb = df[df['month_number'] == i].groupby('person')
    udf_tmp2i = gb.agg({'is_conversion':'sum', 'is_checkout':'sum'})
    udf_tmp2i.columns = ['total_conversions_month_{}'.format(i), 'total_checkouts_month_{}'.format(i)]

    udf_tmp2i['total_checkouts_month_{}'.format(i)] = udf_tmp2i['total_checkouts_month_{}'.format(i)].astype('int')
    udf_tmp2i['total_conversions_month_{}'.format(i)] = udf_tmp2i['total_conversions_month_{}'.format(i)].astype('int')

    udf_tmp2i['has_checkout_month_{}'.format(i)] = udf_tmp2i['total_checkouts_month_{}'.format(i)] > 0
    udf_tmp2i['has_conversion_month_{}'.format(i)] = udf_tmp2i['total_conversions_month_{}'.format(i)] > 0

    udf_tmp2 = udf_tmp2.merge(udf_tmp2i, how='outer', left_index=True, right_index=True)

udf_tmp2 = udf_tmp2.fillna(0)
udf_tmp2 = udf_tmp2.astype('int')
udf_tmp2.head(2)

### 3 - user-not-in-may-features

Checkouts y conversiones por usuario en mayo (último mes registrado)

In [ ]:
# Has conversions or checkouts not in may
gb = df[df['month_number'] != 5].groupby('person')
udf_tmp3 = gb.agg({'is_conversion':'sum', 'is_checkout':'sum'})
udf_tmp3.columns = ['total_conversions_month_1_4', 'total_checkouts_month_1_4']

udf_tmp3['total_checkouts_month_1_4'] = udf_tmp3['total_checkouts_month_1_4'].astype('int')
udf_tmp3['total_conversions_month_1_4'] = udf_tmp3['total_conversions_month_1_4'].astype('int')

udf_tmp3['has_checkout_month_1_4'] = udf_tmp3['total_checkouts_month_1_4'] > 0
udf_tmp3['has_conversion_month_1_4'] = udf_tmp3['total_conversions_month_1_4'] > 0

udf_tmp3 = udf_tmp3.astype('int')
udf_tmp3.head(10)

### 4 - user-lastweek-features

Checkouts y conversiones por usuario en la última semana registrada

In [ ]:
gb = df[df['timestamp'] > pd.to_datetime('2018-05-23')].groupby('person')
udf_tmp4 = gb.agg({'is_conversion':'sum', 'is_checkout':'sum'})
udf_tmp4.columns = ['total_conversions_last_week', 'total_checkouts_last_week']

udf_tmp4['total_checkouts_last_week'] = udf_tmp4['total_checkouts_last_week'].astype('int')
udf_tmp4['total_conversions_last_week'] = udf_tmp4['total_conversions_last_week'].astype('int')

udf_tmp4['has_checkout_last_week'] = udf_tmp4['total_checkouts_last_week'] > 0
udf_tmp4['has_conversion_last_week'] = udf_tmp4['total_conversions_last_week'] > 0

udf_tmp4 = udf_tmp4.astype('int')
udf_tmp4.head(10)

### 5 - user-differentmonths-features

Cuan esparcidas (en meses) estan las conversiones de los usuarios

In [ ]:
udf_tmp5 = udf_tmp2['has_conversion_month_1']
display(len(udf_tmp2))
display(len(udf_tmp5))
for i in range(2,6):
    udf_tmp5 = udf_tmp5 + udf_tmp2['has_conversion_month_{}'.format(i)]
    
udf_tmp5 = udf_tmp5.to_frame()
udf_tmp5.columns = ['amount_of_months_that_has_bought']

for i in range(6):
    print('Users that have bought in {} different months: {}'.format(i, len(udf_tmp5[udf_tmp5['amount_of_months_that_has_bought'] >= i])))

udf_tmp5.head(5)

### 6 - user-time-lapses-features

Tiempo desde últimos eventos

In [ ]:
udf_tmp6 = df.groupby('person').agg({'timestamp':'max'})
#display(udf_tmp6.head())
display(len(udf_tmp6))
udf_tmp6['days_to_last_event'] = pd.to_datetime('2018-06-01').dayofyear - udf_tmp6['timestamp'].dt.dayofyear
"""
udf_tmp7 = df[df['event'] == 'checkout'].groupby('person').agg({'timestamp':'max'})
#display(udf_tmp7.head())
display(len(udf_tmp7))
udf_tmp7['days_to_last_checkout'] = pd.to_datetime('2018-06-01').dayofyear - udf_tmp7['timestamp'].dt.dayofyear


udf_tmp8 = df[df['event'] == 'viewed product'].groupby('person').agg({'timestamp':'max'})
#display(udf_tmp8.head())
display(len(udf_tmp8))
udf_tmp8['days_to_last_viewed_product'] = pd.to_datetime('2018-06-01').dayofyear - udf_tmp8['timestamp'].dt.dayofyear
"""
udf_tmp6.columns = ['timestamp_of_last_event', 'days_to_last_event']
udf_tmp6

---

**Se guarda todo en `user-features.csv`**

In [ ]:
udf = udf_tmp1
udf = udf.merge(udf_tmp2, how='outer', on='person')
udf = udf.merge(udf_tmp3, how='outer', on='person')
udf = udf.merge(udf_tmp4, how='outer', on='person')
udf = udf.merge(udf_tmp5, how='outer', on='person')
udf = udf.merge(udf_tmp6, how='outer', on='person')
udf = udf.fillna(0)
udf = udf.astype('int')
display(len(udf))
udf.head()

In [ ]:
# Confirmamos no haber perdido datos en el medio

display(len(udf))
display(len(df['person'].unique()))

In [ ]:
udf.reset_index().to_csv('data/user-features.csv', index=False)